# Record voice

In [ ]:
import pyaudio 
import wave 

In [ ]:
chunk = 1024  
  
sample_format = pyaudio.paInt16   
chanels = 2
  
smpl_rt = 44400
filename = "audio.m4a"

In [ ]:
def record_voice(seconds):
    pa = pyaudio.PyAudio()   
    
    stream = pa.open(format=sample_format, channels=chanels,  
                    rate=smpl_rt, input=True,  
                    frames_per_buffer=chunk) 
    
    print('Recording...') 
    
    frames = []   
    
    for i in range(0, int(smpl_rt / chunk * seconds)): 
        data = stream.read(chunk) 
        frames.append(data) 
    
    stream.stop_stream() 
    stream.close() 
    
    pa.terminate() 
    
    print('Done !!! ') 
    
    sf = wave.open(filename, 'wb') 
    sf.setnchannels(chanels) 
    sf.setsampwidth(pa.get_sample_size(sample_format)) 
    sf.setframerate(smpl_rt) 
    sf.writeframes(b''.join(frames)) 
    sf.close()

# Transcript voice to .txt

In [ ]:
import requests
import time

In [ ]:
upload_endpoint = 'https://api.assemblyai.com/v2/upload'
transcript_endpoint = 'https://api.assemblyai.com/v2/transcript'

headers_auth_only = {'authorization': 'ac2ab90c5aef43f9aecb78f15a00a1ed'}

headers = {
    "authorization": 'ac2ab90c5aef43f9aecb78f15a00a1ed',
    "content-type": "application/json"
}

CHUNK_SIZE = 5_242_880  # 5MB

In [ ]:
def upload(filename):
    def read_file(filename):
        with open(filename, 'rb') as f:
            while True:
                data = f.read(CHUNK_SIZE)
                if not data:
                    break
                yield data

    upload_response = requests.post(upload_endpoint, headers=headers_auth_only, data=read_file(filename))
    return upload_response.json()['upload_url']

In [ ]:
def transcribe(audio_url):
    transcript_request = {
        'audio_url': audio_url
    }

    transcript_response = requests.post(transcript_endpoint, json=transcript_request, headers=headers)
    return transcript_response.json()['id']

In [ ]:
def poll(transcript_id):
    polling_endpoint = transcript_endpoint + '/' + transcript_id
    polling_response = requests.get(polling_endpoint, headers=headers)
    return polling_response.json()

In [ ]:
def get_transcription_result_url(url, seconds):
    transcribe_id = transcribe(url)
    while True:
        data = poll(transcribe_id)
        if data['status'] == 'completed':
            return data, None
        elif data['status'] == 'error':
            return data, data['error']
            
        print(f"waiting for {seconds * 2} seconds")
        time.sleep(seconds * 2)

In [ ]:
def save_transcript(url, title, seconds):
    data, error = get_transcription_result_url(url, seconds)
    
    if data:
        filename = title + '.txt'
        with open(filename, 'w') as f:
            f.write(data['text'])
        print('Transcript saved')
    elif error:
        print("Error!!!", error)

# Use recording and generate transcription

In [ ]:
seconds = 6

In [ ]:
record_voice(seconds)

In [ ]:
filename = "audio.m4a"
audio_url = upload(filename)

save_transcript(audio_url, 'audio_text', seconds)